In [3]:
import matplotlib.image as mpimg 
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.nn import Linear
from torch.utils.data import Dataset, DataLoader


training_images = np.zeros((87,144,108,3))
test_set_images = np.zeros((23,144,108,3))
train_set_labels = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
test_set_labels = np.array([[0,0,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0]]) 


tempstr = ""
for i in range(0,87):
    tempstr = "Training/" + str(i+1) + ".jpeg"
    image = mpimg.imread(tempstr)
    training_images[i] = image
    
for i in range(1, 24):
    tempstr = "Test/" + str(i+1) + ".jpeg"
    image = mpimg.imread(tempstr)
    test_set_images[i - 1] = image

train_set_flatten = training_images.reshape(training_images.shape[0], -1).T
test_set_flatten = test_set_images.reshape(test_set_images.shape[0], -1).T
train_set = train_set_flatten/255
test_set = test_set_flatten/255


train_set = torch.from_numpy(train_set)
train_set = train_set
train_set_labels = torch.from_numpy(train_set_labels)
test_set_labels = torch.from_numpy(test_set_labels)
test_set = torch.from_numpy(test_set)



train_set = train_set.float()
test_set = test_set.float()
train_set_labels = train_set_labels.float()
test_set_labels = test_set_labels.float()

#standardize dataset here
train_set = (train_set - torch.mean(train_set))/ torch.std(train_set)
test_set = (test_set - torch.mean(test_set))/ torch.std(test_set)

print("Size of training set ", train_set.shape)
print("Size test set", test_set.shape)


Size of training set  torch.Size([46656, 87])
Size test set torch.Size([46656, 23])


In [4]:
class catData(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.length = x.shape[1]
    
    def __getitem__(self, index):
        sample = self.x[:, index], self.y[0][index]
        return sample
    
    def __len__(self):
        return self.length
    
data = catData(train_set, train_set_labels)
train_loader = DataLoader(dataset = data, batch_size = 1) 

In [35]:
class Net(nn.Module):
    def __init__(self, layers, p):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(p = p)
        self.hidden = nn.ModuleList()
        self.sig = nn.Sigmoid()
        for i in range(0, len(layers) - 1):
            self.hidden.append(nn.Linear(layers[i], layers[i + 1]))
            
    def forward(self, activation):
        L = len(self.hidden)
        for i, linear_transform in enumerate(self.hidden):
            if i < L  - 1:
                activation = nn.functional.relu(linear_transform(activation))
                self.dropout(activation)
            else:
                activation = linear_transform(activation)
                activation = self.sig(activation)
                
                
        return activation
    
    
def train(data, model, optimizer, loss_function, epochs):
    progress_list  = [1, 10, 25, 50, 100, 250, 500, 750, 900, 1000, 1250, 1500, 1750, 1900]
    model = model.train()
    for epoch in range(epochs):
        for x,y in data:
            optimizer.zero_grad()
            x = x.unsqueeze(1)
            yhat = model(x)
            loss = loss_function(yhat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if epoch in progress_list:
            print("done with " + str(epoch) + " iterations" )
    return loss


layers = [46656, 3, 3, 1]
layers2 = [46656, 100, 1]
layers3 = [46656, 5, 5, 5, 1]

model = Net(layers, 0.6)
model.eval()

loss_function = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
full_train = train(train_loader, model, optimizer, loss_function, 1000)
predict = model(test_set.T)
print(predict)
predict = predict > 0.7
print(predict)
final = (predict + -1*test_set_labels.T)**2
final = torch.sum(final)
print("The algorithm was correct " +str(round((1 - final.item()/len(predict))*100, 3)) + "% accurate")


"""  
predict = model(train_set.T)
print(predict)
predict = predict > 0.6
final = (predict + -1*train_set_labels.T)**2
final = torch.sum(final)
print("The algorithm was correct " +str(round((1 - final.item()/len(predict))*100, 3)) + "% accurate on the training set")   
    

"""   
#1,2, 11,12,13,14,19,20,21,23
#mistakes - 3,4,5,19,22
#best parameters so far - 1000 epochs
#try adam optimizer, he init, dropout (if really good on training)

done with 1 iterations
done with 10 iterations
done with 25 iterations
done with 50 iterations
done with 100 iterations
done with 250 iterations
done with 500 iterations
done with 750 iterations
done with 900 iterations
tensor([[2.0314e-06],
        [3.7529e-06],
        [2.2572e-04],
        [9.8706e-01],
        [9.8433e-01],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [9.9991e-01],
        [9.9694e-01],
        [9.1918e-04],
        [1.3242e-01],
        [1.3242e-01],
        [1.3242e-01],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [9.8373e-01],
        [9.9198e-01],
        [2.7217e-01],
        [2.7535e-01]], grad_fn=<SigmoidBackward>)
tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        

'  \npredict = model(train_set.T)\nprint(predict)\npredict = predict > 0.6\nfinal = (predict + -1*train_set_labels.T)**2\nfinal = torch.sum(final)\nprint("The algorithm was correct " +str(round((1 - final.item()/len(predict))*100, 3)) + "% accurate on the training set")   \n    \n\n'